In [1]:
#!pip install ipynb --upgrade

In [2]:
#!pip install --upgrade pip

In [3]:
#!pip install keras-tcn
#!pip install tensorflow
#!pip install ipynb
#!pip install plotly

In [4]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

sys.path.append('C:/Users/Alex-/Desktop/Beslutsstödssystem/models-decision-system')
os.chdir("..")
import Decision
import Models

os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [5]:
import os

In [6]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [7]:
settings = load_yaml('mock_settings.yaml')

In [8]:
settings['ensemble']['models'][0]

{'linreg': {'split': {'train': 0.8, 'test': 0.2}}}

In [9]:
dataframe = processing.create_dataframe(settings)

In [10]:
dataset = features.add(dataframe, settings['features'])

In [11]:
dataset

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409
...,...,...,...,...
2019-12-25,1.10944,0.00301,54.215370,1.11012
2019-12-26,1.11012,0.00089,50.207241,1.11713
2019-12-27,1.11713,0.00388,49.232845,1.11813


In [12]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [13]:
linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [14]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [15]:
settings['ensemble']['models'][1]['lstm']['layers']

[{'lstm': {'value': 120, 'activation': 'relu'}},
 {'dropout': {'value': 0.15}},
 {'dense': {'value': 50, 'activation': 'relu'}},
 {'dense': {'value': 1}}]

In [16]:
lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

Epoch 1/7
233/233 [==============================] - 1s 4ms/step - loss: 0.0284 - val_loss: 8.6914e-04
Epoch 2/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0057 - val_loss: 0.0023
Epoch 3/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0046 - val_loss: 8.7527e-04
Epoch 4/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 5/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0030 - val_loss: 0.0011
Epoch 6/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0024 - val_loss: 0.0018
Epoch 7/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0022 - val_loss: 8.2397e-04


In [17]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn import preprocessing
from sklearn import utils

In [18]:
df = pd.read_csv("../MockData.csv")
df_dec = pd.read_csv("../decision.csv")

In [19]:
df_dec = df_dec.set_index("DateTime")
df = df.set_index("DateTime")

In [20]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split

In [21]:
df

,Close,RoC,S%D,PSY12,ASY5
DateTime,,,,,
2010-01-27,0.817467,0.279074,0.239967,0.272727,0.452398
2010-01-28,0.802634,0.256945,0.173515,0.272727,0.376690
2010-01-29,0.779823,0.260848,0.136936,0.272727,0.291014
2010-01-31,0.782677,0.276472,0.108876,0.363636,0.304610
2010-02-01,0.793195,0.281944,0.078999,0.363636,0.385942
...,...,...,...,...,...
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453
2019-12-27,0.175454,0.565530,0.421044,0.818182,0.586624
2019-12-29,0.177701,0.567630,0.451166,0.818182,0.585169


In [22]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day
DateTime,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold
2018-05-23,0.333239,0.299195,NaN,NaN,Hold
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy
...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold


In [23]:
col = "Invest next day"
conditions = [df_dec[col] == "Buy"]
choices = ["1"]
df_dec["buy"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Sell"]
choices = ["1"]
df_dec["sell"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Hold"]
choices = ["1"]
df_dec["hold"] = np.select(conditions, choices, default="0")

In [24]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold,0,0,1
2018-05-23,0.333239,0.299195,NaN,NaN,Hold,0,0,1
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell,0,1,0
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold,0,0,1
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold,0,0,1
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold,0,0,1
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold,0,0,1


In [25]:
df["buy"] = df_dec["buy"]
df["sell"] = df_dec["sell"]
df["hold"] = df_dec["hold"]

In [26]:
df = df.dropna()
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [27]:
df.columns[:5]

Index(['Close', 'RoC', 'S%D', 'PSY12', 'ASY5'], dtype='object')

In [28]:
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [29]:
def foofoofoo(lblz):
    container = []
    for row in lblz:
        word = ''
        for value in row:
            word = word + str(value)
        container.append(word)
    return container

In [30]:
labels = df[["buy", "sell", "hold"]].to_numpy()

In [31]:
labels = foofoofoo(labels)

In [32]:
forecast_days = 1
#df['Label'] = df['Close'].shift(-forecast_days)
df_dec.dropna()
data = df
data = data.dropna(how="any")
x = data[df.columns[:5]].to_numpy()
y = data[df.columns[-4]].to_numpy()
y2 = data[df.columns[-3:]].to_numpy()

## Dataset y


In [33]:
def create_split(x, y):
    tss = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tss.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return x_train, y_train, x_test, y_test

In [34]:
x_train, y_train, x_test, y_test = create_split(x, y)
x_train_2, y_train_2, x_test_2, y_test_2 = create_split(x, y2)

In [35]:
y_test_2 = foofoofoo(y_test_2)
y_train_2 = foofoofoo(y_train_2)

## Dataset y2

In [36]:
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
#OneVsRestClassifier(LinearSVC(random_state=0)).fit(x, y).predict(x)

In [37]:
from sklearn.preprocessing import MinMaxScaler

In [38]:
scaler = MinMaxScaler(feature_range=(0,1))

x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

In [39]:
x = scaler.transform(x)

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

In [40]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from pprint import pprint
from sklearn import metrics

# Gridsearch random forest

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


### Functions

In [41]:
def random_search(settings):
    
    # CREATE THE CORRECT MODEL TYPE
    model = select_model(settings)

    # CREATE THE RANDOMIZED SEARCH CV
    rscv = RandomizedSearchCV(estimator=model,
                                  param_distributions=settings['random_params'],
                                  n_iter=100,
                                  cv = 3,
                                  verbose=2,
                                  random_state=42,
                                  n_jobs=-1)

    # FIT THE RANDOM SEARCH
    rscv.fit(settings["x"], settings["y"])
    
    # SELECT THE BEST ESTIMATOR
    best_random = rscv.best_estimator_

    
    
    return rscvm, rscv.best_params_

In [86]:
def grid_search(model, dataset, settings):

    
    # SELECT MODEL
    model = select_model(settings)

    # CREATE GRID SEARCH CV
    grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2)
    grid_search.fit(dataset["x"], dataset["y"])
    best_grid = grid_search.best_estimator_

    return best_grid

In [87]:
def select_model(settings):
    """Select a classification model type. 
    Supported models:
    Random Forest Classifier (default)"""
    if(settings["model"] == "rfc"):
        model = RandomForestClassifier()
    else:
        model = RandomForestClassifier()
    return model

In [101]:
def grid_search_model(model, dataset, settings):
    """Runs a grid search on the given model to define the best parameters for the model. 
    
    Parameters:
    -----------
    model: sklearn model 
    unfitted model
    dataset: dict
    Dictionary that contains x_train, y_train, features and labels
    
    Returns:
    --------
    model: sklearn model
    Same type of model as the input model, with optimized parameters
    """
    # CREATE GRID SEARCH CV
    grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2)
    
    # FIT GRIDSEARCH
    grid_search.fit(dataset["features"], dataset["labels"])
    
    # PICK OUT THE BEST MODEL FROM GRID SEARCH
    model = grid_search.best_estimator_
    
    # RUN MODEL ON BEST PARAMS   
    model.fit(dataset["x_train"], dataset["y_train"])
    
    # RETURN MODEL WITH BEST PARAMS    
    return model

In [1]:
def evaluate(model, test_features, test_labels, encode=True):
    """Evaluate a classifcation model, for example Random Forest Classification
    
    Parameters:
    model: sklearn classification model
    test_features: np.array of features
    test_labels: np.array of labels 
    encode (default=True): boolean, encodes values to int if set to true
    
    Returns:
    accuracy: float, metrics.accuracy_score
    precision: float, metrics.precision_score
    recall: float, metrics.precision_score
    confusion: 
    """
    # PREDICT
    predictions = model.predict(test_features)
    
    # OPTIONAL ENCODING
    if(encode == True):
        le = preprocessing.LabelEncoder()
        le.fit(test_labels)
        test_labels = le.transform(test_labels)
        predictions = le.transform(predictions)

    # CREATE CONFUSION MATRIX
    confusion = metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2])
    
    # PULL ACCURACY FROM METRICS
    accuracy = metrics.accuracy_score(test_labels, predictions)
    precision = metrics.precision_score(test_labels, predictions, average='micro')
    recall = metrics.recall_score(test_labels, predictions, average='micro')
    #print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    # RETURN THE CREATED SCORES
    return accuracy, confusion

# Random search grid

In [46]:
x = np.concatenate((x_train_2, x_test_2))
y = np.concatenate((y_train_2, y_test_2))

In [47]:
dataset = {"x_train": x_train_2, 
           "y_train": y_train_2,
           "x_test": x_test_2,
           "y_test": y_test_2,
           "features": x,
           "labels": y
          }

## Random forest

In [123]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    'grid_params': { # parameters for grid search (specific to each model)
        'n_estimators': [500,1000,2000], # int
        'max_features': ['log2', 'sqrt'], # {“sqrt”, “log2”}, int or float
        'max_depth': [10,100, None], # int, default=None
        'min_samples_split': [3, 10], # int or float, default=2
        'min_samples_leaf': [2, 4], # int or float, default=1
        'bootstrap': [True, False], # bool, default=True
        "class_weight": ["balanced", None] # {“balanced”, “balanced_subsample”}, dict or list of dicts, default=None
    }
}

In [ ]:
rfc = RandomForestClassifier()
rfc_model = grid_search_model(rfc, dataset, grid_settings)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.2s


In [ ]:
rfc_model.fit(x_train_2, y_train_2)
evaluate(rfc_model, x_test_2, y_test_2)

## Logistical Regression

In [3]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    'grid_params': {
        "C": [1,10,100,1000],
        "penalty": ["l1", "l2"],
        "solver": ["liblinear", "lbfgs", "saga", "sag", "newton-cg"],
        "max_iter": [100,1000, 10000],
        "class_weight": ["balanced", None]
    }
}

In [4]:
logreg = LogisticRegression()
logreg_model = grid_search_model(logreg, dataset, grid_settings)

NameError: name 'LogisticRegression' is not defined

In [5]:
logreg_model.fit(x_train_2, y_train_2)
evaluate(logreg_model, x_test_2, y_test_2)

NameError: name 'logreg_model' is not defined

## SVC

In [114]:
grid_settings = {
    "cv": 5, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "C": [0.1, 1,10,100,1000],
        "kernel": ["linear", "rbf", "sigmoid"]
    }
}

In [115]:
svc = SVC()
svc_model = grid_search_model(svc, dataset, grid_settings)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    2.7s finished


In [116]:
svc_model.fit(x_train_2, y_train_2)
evaluate(svc_model, x_test_2, y_test_2)

(0.7590361445783133,
 0.7590361445783133,
 0.7590361445783133,
 array([[63,  0,  0],
        [10,  0,  0],
        [10,  0,  0]], dtype=int64))

### Create random search

In [ ]:

rfc = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=rfc,
                              param_distributions=random_grid,
                              n_iter=100,
                              cv = 3,
                              verbose=2,
                              random_state=42,
                              n_jobs=-1)

rf_random.fit(x_train_2, y_train_2)

In [ ]:
base_model = RandomForestClassifier(random_state=4)
base_model.fit(x_train_2, y_train_2)
base_accuracy = evaluate(base_model, x_test_2, y_test_2)
print("==========================")
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test_2, y_test_2)

In [ ]:
base_model.n_estimators

In [ ]:
rf_random.best_params_

#### Create gridsearch params from best values in random search

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 100, 110],
    'max_features': [2, 3, 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2,3,4],
    'n_estimators': [1400, 1600, 2500]
}

In [ ]:
#n = np.linspace(0.8 * rf_random.best_params_["n_estimators"], 1.2 * rf_random.best_params_["n_estimators"], num=5)
#ms = np.linspace(0.7 * rf_random.best_params_["min_samples_split"], 1.3 * rf_random.best_params_["min_samples_split"], num=3)
#ml = np.linspace(0.8 * rf_random.best_params_["min_samples_leaf"], 1.2 * rf_random.best_params_["min_samples_leaf"], num=4)
#*md = np.linspace(0.8 * rf_random.best_params_["max_depth"], 1.2 * rf_random.best_params_["max_depth"], num=4)

In [ ]:
# Create the parameter grid based on the results of random search 
# Create a based model

rf = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_2, y_train_2)

In [ ]:
grid_search.fit(x_train_2, y_train_2)

grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test_2, y_test_2)

In [ ]:
best_grid_model = best_grid

In [ ]:
best_grid_model.fit(x_train_2, y_train_2)
model_output = evaluate(best_grid_model, x_test_2, y_test_2)

In [ ]:
model_output

In [ ]:
bootstrap = grid_search.best_params_["bootstrap"]
max_depth = grid_search.best_params_["max_depth"]
max_features = grid_search.best_params_["max_features"]
min_samples_leaf = grid_search.best_params_["min_samples_leaf"]
min_samples_split = grid_search.best_params_["min_samples_split"]
n_estimators = grid_search.best_params_["n_estimators"]

In [ ]:
rfc = RandomForestClassifier(bootstrap=bootstrap, 
                             max_depth=max_depth, 
                             max_features=max_features, 
                             min_samples_leaf=min_samples_leaf,
                            min_samples_split=min_samples_split,
                            n_estimators=n_estimators)
rfc.fit(x_train_2, y_train_2)
model_output = evaluate(rfc, x_test_2, y_test_2)

In [ ]:
model_output[3]

In [ ]:
{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

## random forest

In [ ]:
y_train

In [ ]:


clf = RandomForestClassifier(max_depth=5, 
                             random_state=0, 
                             n_estimators=100,
                             class_weight="balanced"
                             #class_weight="[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 0}]"
                            )
clf.fit(x_train_2,y_train_2)
predict = clf.predict(x_test_2)

In [ ]:
predict

In [ ]:
def svc(x_train, y_train, x_test, y_test):
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 0.06, 'scale'],
                     'C': [1, 10, 15, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000]}]
    scores = ['precision', 'recall']
    
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(
            SVC(max_iter=10000, random_state=5, class_weight="balanced", decision_function_shape='ovo'), 
            tuned_parameters, 
            scoring='%s_macro' % score,
            cv = 5
        )
        clf.fit(x_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(x_test)
        print(classification_report(y_true, y_pred))
        print(y_pred)
        return y_true, y_pred

In [ ]:
y_true, y_pred = svc(x_train, y_train_2, x_test, y_test_2)

In [ ]:
#data_eurusd = pd.read_csv("../EURUSD1m.csv")
data_mock = pd.read_csv("../MockData.csv")

In [ ]:
data_eurusd.head()

In [ ]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [ ]:
os.chdir("..")
import Models